Accent-recognizing model, based on example from https://www.tensorflow.org/tutorials/audio/simple_audio and https://keras.io/examples/audio/speaker_recognition_using_cnn/

Not included:
- Hyperparams tuning
- Underlying Yamnet model

In [1]:
%pip install -U -q tensorflow tensorflow_datasets
#apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
%pip install -U -q keras-tuner

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import pathlib
import os
from os.path import isfile, join, splitext
import librosa
import soundfile as sf
from pydub import AudioSegment
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import tensorflow_io as tfio
import tensorflow_hub as tfhub
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Sequential
import keras_tuner as kt

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Audio

SAMPLING_RATE = 16000
EPOCHS = 50
BATCH_SIZE = 128
SHUFFLE_SEED = 43

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
2023-01-16 23:35:59.818934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Check how many classes are there.

In [2]:
df = pd.read_csv(os.getcwd() + "/data/audio.csv", on_bad_lines = 'skip' , delimiter= ';')
# print(len(df['native_langs'].unique()))
# for lang in df['native_langs'].unique():
#     print(f'"{lang}",')
class_names = df['native_langs'].unique()
lang_idxs = range(len(class_names))
class_dict = dict(zip(class_names, lang_idxs))

print(f"There are {len(class_names)} classes.")
print(f"There are {len(class_dict)} classes.")

There are 228 classes.
There are 228 classes.


### Prepare dataset

In [3]:
def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def dataframe_to_dataset(dataframe):
    """Constructs a dataset of audios and labels."""
    
    # print(dataframe.info())
    # Rewrite file_name to contain file paths
    dataframe['file_name'] = dataframe.apply(
        lambda row: os.path.join(os.getcwd(), 'data/audio_wav', row["file_name"] + ".wav"), 
        axis=1
    )
    # Convert the labels into numbers
    dataframe['native_langs'] = dataframe.apply(
        lambda row: class_dict[row['native_langs']],
        axis=1
    )

    path_ds = tf.data.Dataset.from_tensor_slices(dataframe['file_name'])
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(dataframe['native_langs'])
    
    return tf.data.Dataset.zip((audio_ds, label_ds))    

    # print(len(path_ds), len(audio_ds), len(label_ds))
    # dataframe = df[(df["native_langs"] == "amharic") | (df["native_langs"] == "indonesian")]
    # print(dataframe.shape)



def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

Read the list of audio file paths and labels from the CSV file, as Pandas dataframe.

**IMPORTANT!**
To avoid noises, we are experimenting with only the most frequent classes in our dataset (English and Spanish). In order to work with the entire classes, we are going to identify infrequent classes and remove them from the dataset.

In [4]:
dataframe = pd.read_csv(os.getcwd() + "/data/audio.csv", on_bad_lines = 'skip' , delimiter= ';')
dataframe = dataframe[(dataframe['native_langs'] == 'english') | (dataframe['native_langs'] == 'spanish')]
print(len(dataframe))

886


Split training & validation set.

In [5]:
# Shuffle
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
# rng = np.random.RandomState(SHUFFLE_SEED)
# rng.shuffle(audio_paths)
# rng = np.random.RandomState(SHUFFLE_SEED)
# rng.shuffle(labels)

# Splitting training and validation set
split = int(len(dataframe) * 0.8)
train_df = dataframe[:split]
valid_df = dataframe[split:]

train_ds = dataframe_to_dataset(train_df)
valid_ds = dataframe_to_dataset(valid_df)

train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-16 23:41:06.968474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model definition

In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model = build_model((SAMPLING_RATE // 2, 1), len(class_names))
model.summary()

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)
model_save_filename = "model.h5"

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

### Model training

In [105]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/50
6/6 [==============================] - 44s 5s/step - loss: 3.2613 - accuracy: 0.5353 - val_loss: 1.5944 - val_accuracy: 0.6910
Epoch 2/50
6/6 [==============================] - 29s 4s/step - loss: 1.3340 - accuracy: 0.7458 - val_loss: 0.6525 - val_accuracy: 0.6910
Epoch 3/50
6/6 [==============================] - 27s 4s/step - loss: 0.6259 - accuracy: 0.6992 - val_loss: 0.5930 - val_accuracy: 0.6910
Epoch 4/50
6/6 [==============================] - 28s 4s/step - loss: 0.5710 - accuracy: 0.7458 - val_loss: 0.5629 - val_accuracy: 0.6910
Epoch 5/50
6/6 [==============================] - 30s 4s/step - loss: 0.5407 - accuracy: 0.7458 - val_loss: 0.5585 - val_accuracy: 0.6910
Epoch 6/50
6/6 [==============================] - 32s 4s/step - loss: 0.5323 - accuracy: 0.7458 - val_loss: 0.5570 - val_accuracy: 0.6910
Epoch 7/50
6/6 [==============================] - 29s 4s/step - loss: 0.5364 - accuracy: 0.7458 - val_loss: 0.5676 - val_accuracy: 0.6910
Epoch 8/50
6/6 [==================

In [106]:
print(model.evaluate(valid_ds))

6/6 [==============================] - 3s 319ms/step - loss: 0.5344 - accuracy: 0.7247
[0.5344429016113281, 0.7247191071510315]


### Experiment with data split

When previously we had 80-20 training-validation split, now we are experimenting with 90-10 split.

In [7]:
split = int(len(dataframe) * 0.9)
train_df2 = dataframe[:split]
valid_df2 = dataframe[split:]

train_ds2 = dataframe_to_dataset(train_df2)
valid_ds2 = dataframe_to_dataset(valid_df2)

train_ds2 = train_ds2.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)
valid_ds2 = valid_ds2.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds2 = train_ds2.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
valid_ds2 = valid_ds2.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
train_ds2 = train_ds2.prefetch(tf.data.AUTOTUNE)
valid_ds2 = valid_ds2.prefetch(tf.data.AUTOTUNE)

In [8]:
model2 = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model2.summary()

# Compile the model using Adam's default learning rate
model2.compile(
    optimizer="Adam", 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model2.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_19 (Conv1D)             (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation_13 (Activation)     (None, 8000, 16)     0           ['conv1d_19[0][0]']              
                                                                                                  
 conv1d_20 (Conv1D)             (None, 8000, 16)     784         ['activation_13[0][0]']          
                                                                                            

In [13]:
history = model2.fit(
    train_ds2,
    epochs=EPOCHS,
    validation_data=valid_ds2,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/50
7/7 [==============================] - 46s 5s/step - loss: 0.5269 - accuracy: 0.7415 - val_loss: 0.6664 - val_accuracy: 0.6742
Epoch 2/50
7/7 [==============================] - 33s 4s/step - loss: 0.5498 - accuracy: 0.7415 - val_loss: 0.5792 - val_accuracy: 0.6742
Epoch 3/50
7/7 [==============================] - 30s 4s/step - loss: 0.5259 - accuracy: 0.7415 - val_loss: 0.5848 - val_accuracy: 0.6742
Epoch 4/50
7/7 [==============================] - 30s 4s/step - loss: 0.5332 - accuracy: 0.7415 - val_loss: 0.6056 - val_accuracy: 0.6742
Epoch 5/50
7/7 [==============================] - 31s 4s/step - loss: 0.5260 - accuracy: 0.7390 - val_loss: 0.5981 - val_accuracy: 0.6742
Epoch 6/50
7/7 [==============================] - 31s 4s/step - loss: 0.5092 - accuracy: 0.7428 - val_loss: 0.6173 - val_accuracy: 0.6742
Epoch 7/50
7/7 [==============================] - 30s 4s/step - loss: 0.5148 - accuracy: 0.7415 - val_loss: 0.5907 - val_accuracy: 0.6742
Epoch 8/50
7/7 [==================

In [14]:
print(model2.evaluate(valid_ds2))

3/3 [==============================] - 1s 256ms/step - loss: 0.5743 - accuracy: 0.6854
[0.5742688775062561, 0.6853932738304138]


Experiment with 70-30 split.

In [11]:
split = int(len(dataframe) * 0.9)
train_df3 = dataframe[:split]
valid_df3 = dataframe[split:]

train_ds3 = dataframe_to_dataset(train_df3)
valid_ds3 = dataframe_to_dataset(valid_df3)

train_ds3 = train_ds3.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(BATCH_SIZE)
valid_ds3 = valid_ds3.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds3 = train_ds3.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
valid_ds3 = valid_ds3.map(
    lambda x, y: (audio_to_fft(x), y), 
    num_parallel_calls=tf.data.AUTOTUNE,
)
train_ds3 = train_ds3.prefetch(tf.data.AUTOTUNE)
valid_ds3 = valid_ds3.prefetch(tf.data.AUTOTUNE)

In [12]:
model3 = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model3.summary()

# Compile the model using Adam's default learning rate
model3.compile(
    optimizer="Adam", 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model3.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_37 (Conv1D)             (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation_26 (Activation)     (None, 8000, 16)     0           ['conv1d_37[0][0]']              
                                                                                                  
 conv1d_38 (Conv1D)             (None, 8000, 16)     784         ['activation_26[0][0]']          
                                                                                            

In [16]:
history = model3.fit(
    train_ds3,
    epochs=EPOCHS,
    validation_data=valid_ds3,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/50
7/7 [==============================] - 40s 4s/step - loss: 2.8689 - accuracy: 0.5596 - val_loss: 1.9253 - val_accuracy: 0.6742
Epoch 2/50
7/7 [==============================] - 32s 4s/step - loss: 1.0548 - accuracy: 0.6060 - val_loss: 0.6417 - val_accuracy: 0.6742
Epoch 3/50
7/7 [==============================] - 32s 4s/step - loss: 0.6066 - accuracy: 0.7415 - val_loss: 0.6134 - val_accuracy: 0.6742
Epoch 4/50
7/7 [==============================] - 31s 4s/step - loss: 0.5825 - accuracy: 0.7177 - val_loss: 0.6336 - val_accuracy: 0.6742
Epoch 5/50
7/7 [==============================] - 31s 4s/step - loss: 0.5446 - accuracy: 0.7415 - val_loss: 0.5782 - val_accuracy: 0.6742
Epoch 6/50
7/7 [==============================] - 32s 4s/step - loss: 0.5284 - accuracy: 0.7415 - val_loss: 0.6403 - val_accuracy: 0.6742
Epoch 7/50
7/7 [==============================] - 38s 5s/step - loss: 0.5465 - accuracy: 0.7415 - val_loss: 0.5801 - val_accuracy: 0.6742
Epoch 8/50
7/7 [==================

In [17]:
print(model3.evaluate(valid_ds3))

3/3 [==============================] - 1s 206ms/step - loss: 0.5740 - accuracy: 0.6854
[0.574015736579895, 0.6853932738304138]
